In [1]:
import pandas as pd
from api import Features
from model import ModelWithScoring
import joblib

### Результат prob'ы должен совпасть с использованием интерфейса модели на прямую (predict_proba())

In [2]:
test = pd.read_csv('cut_test.csv')
model = joblib.load('lightgbm_optuna_model_cut.pkl')

/Users/vi/DataspellProjects/shift/.venv/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.3.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
x_test=test.drop(['target', 'sk_id_curr'], axis=1)
y_test = test['target']
sub_test = pd.DataFrame(test['sk_id_curr'].copy())

In [4]:
sub_test['lgbm']=model.predict_proba(x_test)[:, 1] 

In [5]:
model_core = ModelWithScoring('lightgbm_optuna_model_cut.pkl')
x_test['result'] = x_test.apply(lambda x: model_core.get_scoring_result(Features(*x)),axis=1,)
sub_test['lgbm_core'] = x_test['result'].apply(lambda x: x.proba)
print(sub_test.head())

   sk_id_curr      lgbm  lgbm_core
0      146856  0.229928   0.229928
1      285764  0.369700   0.369700
2      150665  0.653448   0.653448
3      305131  0.493769   0.493769
4      376689  0.165342   0.165342


/Users/vi/DataspellProjects/shift/.venv/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.3.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


### Вывод решения по сумме выдачи и расчет прибыли

In [6]:
x_test=test.drop(['target', 'sk_id_curr'], axis=1)
y_test = test['target']

In [7]:
x_test['result'] = x_test.apply(lambda x: model_core.get_scoring_result(Features(*x)),axis=1,)
x_test['decision'] = x_test['result'].apply(lambda x: x.decision.name)
x_test['amount'] = x_test['result'].apply(lambda x: x.amount)
x_test['proba'] = x_test['result'].apply(lambda x: x.proba)

In [8]:
decisions=pd.concat([test['sk_id_curr'], x_test[['decision', 'amount', 'proba']]], axis=1)
print(decisions.head())

   sk_id_curr  decision  amount     proba
0      146856  accepted  190000  0.229928
1      285764  declined       0  0.369700
2      150665  declined       0  0.653448
3      305131  declined       0  0.493769
4      376689  accepted  365000  0.165342


In [9]:
result = decisions.groupby(['decision', 'amount']).agg({'proba': 'mean', 'sk_id_curr': 'count'}).rename(columns={'sk_id_curr': 'count'}).reset_index()
print(result)

    decision  amount     proba  count
0   accepted   25000  0.225450      7
1   accepted   50000  0.171333      6
2   accepted   65000  0.192477      6
3   accepted   90000  0.147886      2
4   accepted  100000  0.155722      2
5   accepted  125000  0.226883      1
6   accepted  150000  0.198043      2
7   accepted  185000  0.245555      1
8   accepted  190000  0.181897      5
9   accepted  225000  0.184460      3
10  accepted  325000  0.193247      5
11  accepted  350000  0.174115      2
12  accepted  365000  0.145812      3
13  accepted  390000  0.115624      1
14  accepted  400000  0.242209      1
15  accepted  450000  0.094770      1
16  accepted  490000  0.150292      2
17  accepted  525000  0.155067      1
18  accepted  590000  0.071338      1
19  accepted  650000  0.088938      1
20  accepted  765000  0.095573      1
21  accepted  890000  0.091048      1
22  declined       0  0.494620    145


In [10]:
result['default'] = round(result['count'] * result['proba'])
result['return'] = result['count'] - result['default']
result['loss'] = result['default'] * result['amount']
result['revenue'] = result['return'] * result['amount']*0.1
result['profit'] = result['revenue'] - result['loss']
print(result)

    decision  amount     proba  count  default  return      loss   revenue  \
0   accepted   25000  0.225450      7      2.0     5.0   50000.0   12500.0   
1   accepted   50000  0.171333      6      1.0     5.0   50000.0   25000.0   
2   accepted   65000  0.192477      6      1.0     5.0   65000.0   32500.0   
3   accepted   90000  0.147886      2      0.0     2.0       0.0   18000.0   
4   accepted  100000  0.155722      2      0.0     2.0       0.0   20000.0   
5   accepted  125000  0.226883      1      0.0     1.0       0.0   12500.0   
6   accepted  150000  0.198043      2      0.0     2.0       0.0   30000.0   
7   accepted  185000  0.245555      1      0.0     1.0       0.0   18500.0   
8   accepted  190000  0.181897      5      1.0     4.0  190000.0   76000.0   
9   accepted  225000  0.184460      3      1.0     2.0  225000.0   45000.0   
10  accepted  325000  0.193247      5      1.0     4.0  325000.0  130000.0   
11  accepted  350000  0.174115      2      0.0     2.0       0.0

In [11]:
result["profit"].sum()

258500.0